In [126]:
import requests
import json
import pandas as pd

In [103]:
base_url = 'https://cloud.iexapis.com/stable' # <- single HTTP request. What does batch allow?
token = 'pk_8d0f8e5a2a134ad48410c868b4849d70'

In [104]:
pract = requests.get(base_url + '/stock/market/batch?symbols=mmm,amzn,goog&types=dividends,stats&filter=marketcap,amount&range=1y&token=' + token).json()

In [105]:
pract

{'MMM': {'stats': {'marketcap': 85474125600},
  'dividends': [{'amount': '1.44', 'date': '2020-05-02'}]},
 'AMZN': {'stats': {'marketcap': 1139648091000}, 'dividends': []},
 'GOOG': {'stats': {'marketcap': 901474126010}, 'dividends': []}}

In [72]:
requests.get(base_url + '/stock/mmm/dividends/3m?token=' + token).json()

[{'exDate': '2020-02-13',
  'paymentDate': '2020-03-12',
  'recordDate': '2020-02-14',
  'declaredDate': '2020-02-04',
  'amount': '1.47',
  'flag': 'Cash',
  'currency': 'USD',
  'description': 'Ordinary Shares',
  'frequency': 'quarterly',
  'date': '2020-05-01'}]

In [92]:
# Pull recent data. Look at how companies performed since onset of Corona. Can pull market data, then group into cap weighting. 
# What else? Stock price change? 
aapl_price_march_2 = requests.get(base_url + '/stock/aapl/chart/date/20200302?filter=close&chartByDay=true&token=' + token).json()[0]['close']

In [93]:
aapl_price_april_30 = requests.get(base_url + '/stock/aapl/quote?filter=latestPrice&token=' + token).json()['latestPrice']

In [101]:
percent_change = ((aapl_price_april_30- aapl_price_march_2) / aapl_price_march_2) * 100
round(percent_change,2)

-1.68

In [5]:
# What are different endpoints for IEX cloud?
# How can multiple API calls be put in one HTTP request?

In [120]:
# Testing request parameter. 
# Variables to get: March 2nd stock price. Latest Price. Industry ('Company' endpoint). Dividends ('Dividends Basic'). Market Share ('Stats')
requests.get(base_url + '/stock/aapl/batch?types=chart,quote&range=date&exactDate=20200302&chartCloseOnly=true&chartByDay=true&filter=chart,latestPrice&token=' + token).json()
# March 2nd stock price and latest price work as above. 

{'chart': [{'date': '2020-03-02',
   'uClose': 298.81,
   'uOpen': 282.28,
   'uHigh': 301.44,
   'uLow': 277.72,
   'uVolume': 85349339,
   'close': 298.81,
   'open': 282.28,
   'high': 301.44,
   'low': 277.72,
   'volume': 85349339,
   'currency': '',
   'change': 25.45,
   'changePercent': 9.3101,
   'label': 'Mar 2, 20',
   'changeOverTime': 0.093101}],
 'quote': {'latestPrice': 289.07}}

In [116]:
# Adding Industry, Dividends. Scraping dividends. Add marketshare
requests.get(base_url + '/stock/aapl/batch?types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token=' + token).json()
# Secured bag for one company. Can we do it with multiple?

{'chart': [{'date': '2020-03-02',
   'uClose': 298.81,
   'uOpen': 282.28,
   'uHigh': 301.44,
   'uLow': 277.72,
   'uVolume': 85349339,
   'close': 298.81,
   'open': 282.28,
   'high': 301.44,
   'low': 277.72,
   'volume': 85349339,
   'currency': '',
   'change': 25.45,
   'changePercent': 9.3101,
   'label': 'Mar 2, 20',
   'changeOverTime': 0.093101}],
 'quote': {'latestPrice': 289.07},
 'company': {'industry': 'Telecommunications Equipment'},
 'stats': {'marketcap': 1264820003600}}

In [121]:
# Trying to fetch multiple symbols
aapl_amazn_goog_pract = requests.get(base_url + '/stock/market/batch?symbols=aapl,amzn,goog&types=chart,quote,company,stats&range=day&exactDate=20200302&chartByDay=true&filter=industry,chart,latestPrice,marketcap&token=' + token).json()
# Success! how many messages will it cost??
# Add shares outstanding.

In [131]:
# Turn returned JSON object into a dataframe? 

pd.DataFrame(aapl_amazn_goog_pract).T

,company,stats,quote,chart
AAPL,{'industry': 'Telecommunications Equipment'},{'marketcap': 1264820003600},{'latestPrice': 289.07},"[{'date': '2020-03-02', 'uClose': 298.81, 'uOp..."
AMZN,{'industry': 'Internet Retail'},{'marketcap': 1139648091000},{'latestPrice': 2286.04},"[{'date': '2020-03-02', 'uClose': 1953.95, 'uO..."
GOOG,{'industry': 'Internet Software/Services'},{'marketcap': 901474126010},{'latestPrice': 1320.61},"[{'date': '2020-03-02', 'uClose': 1389.11, 'uO..."


In [132]:
# Create function that will clean the DataFrame constructed from returned IEX JSON. 
def clean_json_df(row):
    row['company'] = row['company']['industry']
    row['stats'] = row['stats']['marketcap']
    row['quote'] = row['quote']['latestPrice']
    row['chart'] = row['chart'][0]['close']
    return row


In [137]:
pd.DataFrame(aapl_amazn_goog_pract).T.apply(clean_json_df, axis = 1).rename(columns = lambda x: x.title())

,Company,Stats,Quote,Chart
AAPL,Telecommunications Equipment,1264820003600,289.07,298.81
AMZN,Internet Retail,1139648091000,2286.04,1953.95
GOOG,Internet Software/Services,901474126010,1320.61,1389.11


In [48]:
# Sandbox testing
sb_base_url = 'https://sandbox.iexapis.com/stable'
sb_token = 'Tpk_8ef08bcd612444eab903cf6d1877b2bb'
ticker = 'F'

In [33]:
# data-points endpoint is not working
requests.get('https://sandbox.iexapis.com/stable/data-points/aapl?token=Tpk_8ef08bcd612444eab903cf6d1877b2bb').json()

[{'key': 'NEXTDIVIDENDDATE',
  'weight': 1,
  'description': '',
  'lastUpdated': '2020-02-07T16:24:55+00:00'},
 {'key': 'ACCOUNTSPAYABLE',
  'weight': 3000,
  'description': 'Balance Sheet: accountsPayable',
  'lastUpdated': '2020-04-18T08:07:58+00:00'},
 {'key': 'CAPITALSURPLUS',
  'weight': 3000,
  'description': 'Balance Sheet: capitalSurplus',
  'lastUpdated': '2020-04-18T08:07:58+00:00'},
 {'key': 'COMMONSTOCK',
  'weight': 3000,
  'description': 'Balance Sheet: commonStock',
  'lastUpdated': '2020-04-18T08:07:58+00:00'},
 {'key': 'CURRENTASSETS',
  'weight': 3000,
  'description': 'Balance Sheet: currentAssets',
  'lastUpdated': '2020-04-18T08:07:58+00:00'},
 {'key': 'CURRENTCASH',
  'weight': 3000,
  'description': 'Balance Sheet: currentCash',
  'lastUpdated': '2020-04-18T08:07:58+00:00'},
 {'key': 'CURRENTLONGTERMDEBT',
  'weight': 3000,
  'description': 'Balance Sheet: currentLongTermDebt',
  'lastUpdated': '2020-04-18T08:07:58+00:00'},
 {'key': 'GOODWILL',
  'weight': 3000,

In [46]:
key = 'ENTERPRISEVALUE'
requests.get(sb_base_url + '/data-points/aapl/' + key + '?token=' + sb_token).text

'1305116362054'

In [54]:
requests.get(sb_base_url + '/files/info/:id' + '?token=' + sb_token).json()

{'symbols': [], 'dates': []}

In [60]:
requests.get(base_url + '/files' + '?token=' + token).text

'The requested data is not available to free tier accounts. Please upgrade for access to this data.'

In [65]:
requests.get(base_url + '/time-series?token=' + token).json()

[{'id': 'PREMIUM_WALLSTREETHORIZON_OPTION_EXPIRATION_DATE',
  'description': 'Company Events (Option Expiration Date)',
  'schema': {'type': 'object',
   'properties': {'eventId': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'optionSymbol': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'symbol': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'companyName': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'usStockExchange': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'expirationDate': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'expirationFrequency': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'strikeStatus': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'optionType': {'anyOf': [{'type': 'string'}, {'type': 'null'}]},
    'updated': {'anyOf': [{'type': 'string'}, {'type': 'null'}]}},
   'required': [],
   'additionalProperties': True},
  'weightKey': 'PREMIUM_WALLSTREETHORIZON_OPTION_EXPIRATION_DATE',
  'weight':

In [71]:
requests.get(sb_base_url + '/time-series/REPORTED_FINANCIALS/AAPL?token=' + sb_token).json()

[{'AccountsPayableCurrent': 47474479474,
  'formFiscalYear': 2111,
  'version': 'guaaps-',
  'periodStart': 1570595017945,
  'periodEnd': 1600472012488,
  'dateFiled': 1584602258508,
  'formFiscalQuarter': None,
  'reportLink': 'oa0033tcds//:tgre./9ist1A/p0a1h1/ge9w0/d/evvca2/wsr0900wh000109203103.',
  'AccountsReceivableNetCurrent': 23205167966,
  'AccruedIncomeTaxesNoncurrent': 30820885219,
  'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment': 59107365619,
  'AccumulatedOtherComprehensiveIncomeLossNetOfTax': -606601018,
  'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensationIncludingTransferPricing': 628038172,
  'AllocatedShareBasedCompensationExpense': 6190235210,
  'AmountUtilizedUnderShareRepurchaseProgram': 100073194685,
  'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount': 16166447,
  'Assets': 339138777087,
  'AssetsCurrent': 170084669830,
  'AssetsNoncurrent': 184371352639,
  'AvailableForSaleDebtSecuritiesAccumu

In [75]:
requests.get(base_url + '/stock/F/quote?token=' + token).json()

{'symbol': 'F',
 'companyName': 'Ford Motor Co.',
 'primaryExchange': 'New York Stock Exchange',
 'calculationPrice': 'close',
 'open': 5.15,
 'openTime': 1587130201107,
 'openSource': 'official',
 'close': 5.12,
 'closeTime': 1587153720895,
 'closeSource': 'official',
 'high': 5.28,
 'highTime': 1587153338149,
 'highSource': '15 minute delayed price',
 'low': 5.03,
 'lowTime': 1587134034102,
 'lowSource': '15 minute delayed price',
 'latestPrice': 5.12,
 'latestSource': 'Close',
 'latestTime': 'April 17, 2020',
 'latestUpdate': 1587153720895,
 'latestVolume': 91840596,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 5.125,
 'delayedPriceTime': 1587153338149,
 'oddLotDelayedPrice': 5.126,
 'oddLotDelayedPriceTime': 1587153338210,
 'extendedPrice': 5.08,
 'extendedChange': -0.04,
 'extendedChangePercent': -0.00781,
 'extendedPriceTime': 1587167998137,
 'previousClose': 4.94,
 'previousVolume': 58951213,
 'change': 0.18,
 'changePercent': 0.

In [85]:
requests.get(base_url + '/stock/market/batch/?symbols=aapl,f&types=ceo-compensation&token=' + token).text

'The requested data is not available to free tier accounts. Please upgrade for access to this data.'